I suppose the easiest way of approaching this is just to see which is closest after a gazillion or so ticks. I kind of like the idea of storing the input as a `pd.DataFrame` in the first instance:

In [1]:
import pandas as pd
import re

import itertools as it

In [2]:
testInput_str='''
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>
'''

To parse the input, let's assume that each line of the input contains exactly 9 distinct integers, and put those integers into columns:

In [3]:

state_ls=[[] for i in range(9)]

for nl in testInput_str.strip().split('\n'):
    values_ls=re.findall('\-?\d+', nl)
    assert len(values_ls)==9
    for (i,v) in enumerate(values_ls):
        state_ls[i].append(int(v))

state_df=pd.DataFrame(state_ls)
state_df=state_df.T
state_df.columns=['px', 'py', 'pz', 'vx', 'vy', 'vz', 'ax', 'ay', 'az']

# make a copy for future reference:
testState_df=state_df.copy()

state_df

,px,py,pz,vx,vy,vz,ax,ay,az
0,3,0,0,2,0,0,-1,0,0
1,4,0,0,0,0,0,-2,0,0


Now that we're in a dataframe, everything's vectorised. So for a single step, we just need to do:

In [4]:
def tick(state_df):
    '''
    Horribly use the mutability of a dataframe in a function
    to update state_df to the state after one tick.
    '''
    # First update the velocities:
    state_df['vx']=state_df['vx']+state_df['ax']
    state_df['vy']=state_df['vy']+state_df['ay']
    state_df['vz']=state_df['vz']+state_df['az']
    
    # And then update the positions:
    state_df['px']=state_df['px']+state_df['vx']
    state_df['py']=state_df['py']+state_df['vy']
    state_df['pz']=state_df['pz']+state_df['vz']
    

Check that that's behaving itself:

In [5]:

state_df=testState_df.copy()

assert list(state_df.loc[0][['px', 'py', 'pz']])==[3,0,0]
assert list(state_df.loc[1][['px', 'py', 'pz']])==[4,0,0]

tick(state_df)

assert list(state_df.loc[0][['px', 'py', 'pz']])==[4,0,0]
assert list(state_df.loc[1][['px', 'py', 'pz']])==[2,0,0]

tick(state_df)

assert list(state_df.loc[0][['px', 'py', 'pz']])==[4,0,0]
assert list(state_df.loc[1][['px', 'py', 'pz']])==[-2,0,0]

tick(state_df)

assert list(state_df.loc[0][['px', 'py', 'pz']])==[3,0,0]
assert list(state_df.loc[1][['px', 'py', 'pz']])==[-8,0,0]



Good. Now "just" need to work out which particle stays closest in the long term.

For a given `state_df`, we can add the position columns for the manhattan distance, and then use the `.idxmin()` method to find which is the current closest state:

In [6]:
(abs(state_df['px'])+abs(state_df['py'])+abs(state_df['pz'])).idxmin()

0

So let's be hacky, and just run the puzzle input until it settles down.

First build the state using the puzzle input:

In [7]:
with open('data/day20.txt') as fIn:
    puzzleInput_str=fIn.read()

state_ls=[[] for i in range(9)]

for nl in puzzleInput_str.strip().split('\n'):
    values_ls=re.findall('\-?\d+', nl)
    assert len(values_ls)==9
    for (i,v) in enumerate(values_ls):
        state_ls[i].append(int(v))

puzzleState_df=pd.DataFrame(state_ls)
puzzleState_df=puzzleState_df.T
puzzleState_df.columns=['px', 'py', 'pz', 'vx', 'vy', 'vz', 'ax', 'ay', 'az']
puzzleState_df.head()

,px,py,pz,vx,vy,vz,ax,ay,az
0,-717,-4557,2578,153,21,30,-8,8,-7
1,1639,651,-987,29,-19,129,-5,0,-6
2,-10482,-248,-491,4,10,81,21,0,-4
3,-6607,-2542,1338,-9,52,-106,14,2,4
4,-4468,1178,-6474,146,44,66,0,-5,9


And now let's hackily go through the ticks until it all seems to have settled down. I'll print the closest particle every time it changes:

In [8]:
state_df=puzzleState_df.copy()

totalIterations_i=10000
closestTally_idx=-1

for i in range(totalIterations_i):
    closest_idx=(abs(state_df['px'])+abs(state_df['py'])+abs(state_df['pz'])).idxmin()
    if not closest_idx==closestTally_idx:
        closestTally_idx=closest_idx
        print('{}\t{}'.format(i, closestTally_idx))
    tick(state_df)


0	55
1	218
3	255
8	129
10	467
11	89
12	51
13	55
14	56
15	211
16	135
17	371
18	7
19	107
20	175
21	301
22	310
23	215
24	159
25	237
26	557
27	540
28	390
29	117
30	530
31	74
32	229
33	92
34	264
35	524
36	82
37	199
38	407
39	144
40	370
41	550
49	14
309	344


I'm happy to go with that 344.